In [89]:
### Prepare data for serve

In [91]:
import random
import joblib
import numpy as np
import pandas as pd
pd.set_option('display.max.column', 500)

In [92]:
books = pd.read_csv('books.csv')

In [93]:
ratings = pd.read_csv('ratings.csv')
ratings

,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3
...,...,...,...
5976474,49925,510,5
5976475,49925,528,4
5976476,49925,722,4
5976477,49925,949,5


In [94]:
ratings = ratings.reset_index()
ratings['implicit_ratings'] = ratings.rating.apply(lambda x: 1 if x>=5 else 0)

In [95]:
ratings['rank_number'] = ratings.sort_values(['user_id', 'index']) \
             .groupby(['user_id']) \
             .cumcount() + 1

In [96]:
ratings = ratings.merge(ratings.user_id.value_counts().reset_index())
ratings['fraction'] = ratings['rank_number']/ratings['count']

In [97]:
train_df = ratings[ratings['fraction']<=0.75]
test_df = ratings[ratings['fraction']>0.75]

In [98]:
train_df

,index,user_id,book_id,rating,implicit_ratings,rank_number,count,fraction
0,0,1,258,5,1,1,117,0.008547
1,75,1,268,3,0,2,117,0.017094
2,76,1,5556,3,0,3,117,0.025641
3,77,1,3638,3,0,4,117,0.034188
4,78,1,1796,5,1,5,117,0.042735
...,...,...,...,...,...,...,...,...
5976449,5976423,49802,6731,3,0,70,99,0.707071
5976450,5976424,49802,6864,3,0,71,99,0.717172
5976451,5976425,49802,8050,3,0,72,99,0.727273
5976452,5976426,49802,7576,3,0,73,99,0.737374


In [99]:
### Make features for ranking

In [100]:
books = pd.read_csv('books.csv')
books.head(10)

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,eng,4.44,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,"Twilight (Twilight, #1)",en-US,3.57,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,To Kill a Mockingbird,eng,4.25,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,The Great Gatsby,eng,3.89,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...
5,6,11870085,11870085,16827462,226,525478817,9.780525e+12,John Green,2012.0,The Fault in Our Stars,The Fault in Our Stars,eng,4.26,2346404,2478609,140739,47994,92723,327550,698471,1311871,https://images.gr-assets.com/books/1360206420m...,https://images.gr-assets.com/books/1360206420s...
6,7,5907,5907,1540236,969,618260307,9.780618e+12,J.R.R. Tolkien,1937.0,The Hobbit or There and Back Again,The Hobbit,en-US,4.25,2071616,2196809,37653,46023,76784,288649,665635,1119718,https://images.gr-assets.com/books/1372847500m...,https://images.gr-assets.com/books/1372847500s...
7,8,5107,5107,3036731,360,316769177,9.780317e+12,J.D. Salinger,1951.0,The Catcher in the Rye,The Catcher in the Rye,eng,3.79,2044241,2120637,44920,109383,185520,455042,661516,709176,https://images.gr-assets.com/books/1398034300m...,https://images.gr-assets.com/books/1398034300s...
8,9,960,960,3338963,311,1416524797,9.781417e+12,Dan Brown,2000.0,Angels & Demons,"Angels & Demons (Robert Langdon, #1)",en-CA,3.85,2001311,2078754,25112,77841,145740,458429,716569,680175,https://images.gr-assets.com/books/1303390735m...,https://images.gr-assets.com/books/1303390735s...
9,10,1885,1885,3060926,3455,679783261,9.780680e+12,Jane Austen,1813.0,Pride and Prejudice,Pride and Prejudice,eng,4.24,2035490,2191465,49152,54700,86485,284852,609755,1155673,https://images.gr-assets.com/books/1320399351m...,https://images.gr-assets.com/books/1320399351s...


In [101]:
def simple_one_hot(genre_dict, genre):
    if genre in genre_dict: 
        return 1
    return 0

In [102]:
genres_df = pd.read_json('goodreads_book_genres_initial.json', lines=True)
genres_df = genres_df[genres_df.book_id.isin(books.goodreads_book_id)]
genres_df.columns = ['book_id', 'genres_dict']
genres_df.head(5)
all_genres = set()

for dict_genre in genres_df.genres_dict: 
    for elem in list(dict_genre.keys()):
        all_genres.add(elem)

all_genres
for genre in all_genres: 
    genres_df[genre] = 0

def simple_one_hot(genre_dict, genre):
    if genre in genre_dict: 
        return 1
    return 0

for genre in all_genres: 
    genres_df[genre] = genres_df.apply(lambda df: simple_one_hot(df['genres_dict'], genre), axis=1)

books_profile = books[['book_id', 'goodreads_book_id']]
train_df = train_df.merge(books_profile, left_on='book_id', right_on='book_id', how='left')
books_profile = books_profile.merge(genres_df, left_on='goodreads_book_id', right_on='book_id', how='left')
train_df = train_df.merge(genres_df, left_on='goodreads_book_id', right_on='book_id', how='left')
train_df.head(5)
users_profiles = train_df.groupby('user_id')[list(all_genres)].sum()
users_profiles.columns = ['user_'+name for name in list(users_profiles)]

train_df.columns = ['book_'+item if item in all_genres else item for item in list(train_df)]
books_profile.columns = ['book_'+item if item in all_genres else item for item in list(books_profile)]




In [103]:
import spacy
nlp = spacy.load("en_core_web_sm")
books['vector_representation'] = books.title.apply(lambda x: nlp(x).vector)
new_vectors_columns_name = [f'vec_component_{num}' for num in range(96)]
books[new_vectors_columns_name] = pd.DataFrame(books.vector_representation.tolist(), index= books.index)


train_df = train_df.merge(books[new_vectors_columns_name+['book_id']], left_on='book_id_x', right_on='book_id')
books_profile = books_profile.merge(books[new_vectors_columns_name+['book_id']], left_on='book_id_x', right_on='book_id')


train_df = train_df.merge(users_profiles, left_on='user_id', right_on='user_id', how='left')
train_df.head(5)
from sklearn.ensemble import RandomForestClassifier



In [104]:
cols_for_using = [
 'book_mystery, thriller, crime',
 'book_non-fiction',
 'book_romance',
 'book_fantasy, paranormal',
 'book_poetry',
 'book_fiction',
 'book_young-adult',
 'book_history, historical fiction, biography',
 'book_comics, graphic',
 'book_children',
 'user_mystery, thriller, crime',
 'user_non-fiction',
 'user_romance',
 'user_fantasy, paranormal',
 'user_poetry',
 'user_fiction',
 'user_young-adult',
 'user_history, historical fiction, biography',
 'user_comics, graphic',
 'user_children']+new_vectors_columns_name
train_df.fillna(0, inplace=True)
classifier = RandomForestClassifier(n_estimators = 20, criterion = 'entropy', random_state = 42)

classifier.fit(train_df[cols_for_using], train_df['implicit_ratings'])

RandomForestClassifier(criterion='entropy', n_estimators=20, random_state=42)

In [105]:
joblib.dump(books_profile, 'books_profile.joblib') 
joblib.dump(users_profiles.reset_index(), 'user_profile.joblib')
joblib.dump(cols_for_using, 'cols_for_using.joblib')
joblib.dump(classifier, 'rf_classifier.joblib')

['rf_classifier.joblib']

In [106]:
import scipy.sparse as sparse
from pandas.api.types import CategoricalDtype 

user_index = train_df.user_id.unique()
books_index = train_df.book_id.unique()

rows = train_df['user_id'].astype(CategoricalDtype(categories=user_index)).cat.codes 
cols = train_df['book_id'].astype(CategoricalDtype(categories=books_index)).cat.codes 

matrix = sparse.csr_matrix((train_df.implicit_ratings, (rows, cols)), shape=(len(user_index), len(books_index)))

matrix = matrix.toarray()

In [107]:
joblib.dump(matrix, 'user_item_matrix.joblib')
joblib.dump(books_index, 'book_index.joblib')
joblib.dump(user_index, 'user_index.joblib')

['user_index.joblib']

In [108]:
for ind, score in enumerate(matrix[50]): 
    if score>=1:
        book_id = books_index[ind]
        print(score, books[books['book_id']==book_id].original_title.values[0])


1 The Book Thief
1 Ender's Game
1 Nineteen Eighty-Four
1 The Giving Tree
1 Matilda
1 The Prophet
1 Der Prozeß
1 Watchmen
1 The Amazing Adventures of Kavalier & Clay
1 nan
1 Briefe an einen jungen Dichter
1 Persepolis
1 From the Mixed-Up Files of Mrs. Basil E. Frankweiler
1 Consider the Lobster
1 Jimmy Corrigan: The Smartest Kid on Earth
1 Random Family: Love, Drugs, Trouble, and Coming of Age in the Bronx


In [109]:
most_popular_book = list(ratings.book_id.value_counts().reset_index()[:40].book_id)
joblib.dump(most_popular_book, 'most_popular_book.joblib')

['most_popular_book.joblib']

In [110]:
read_history_df = train_df[['user_id','book_id_x']].groupby('user_id').agg({"book_id_x":list})
read_history_df = read_history_df.reset_index()
read_history_dict = dict(zip(read_history_df.user_id, read_history_df.book_id_x))
joblib.dump(read_history_dict, 'read_history_dict.joblib')

['read_history_dict.joblib']

In [111]:
import implicit
model = implicit.als.AlternatingLeastSquares(factors=128, iterations=120)
model.fit(sparse.csr_matrix(matrix))

  0%|          | 0/120 [00:00<?, ?it/s]

In [112]:
joblib.dump(model, 'als_model.joblib')

['als_model.joblib']

### AB test simulation

In [113]:
# соберем статистику для модели "самые популярные книги"

In [228]:
import requests
already_checked = set()
conversion_list = []
while len(already_checked)<1000: 
    user_id = random.randint(1, 53424)
    if user_id in already_checked:
        continue
    else: 
        already_checked.add(user_id)
        recommendatations = requests.post('http://127.0.0.1:5509/recommender', json={"model": "most_popular_model", "user_id":user_id})
        recommendatations = recommendatations.json()['result']
        true_books = list(test_df[(test_df.user_id==user_id)&(test_df.implicit_ratings==1)].book_id.values)
        conversion_for_user = len([book for book in recommendatations if book in true_books])/len(recommendatations)
        conversion_list.append(conversion_for_user)    

print(np.mean(conversion_list))




0.020872635809977745


In [182]:
# Текущая конверсия 2.0%, предполагаем что модель улучшит ее до 3.0%,
#  и подсчитаем сколько необходимо собрать пользователей для подсчета значимого результата
# мощность - 80%, уровень значимости  0.95
# https://www.evanmiller.org/ab-testing/sample-size.html дает размер выборки в 3.5 пользователей

In [229]:
# проведем AA тест 

conversion_list_group_a = []
stats_for_group_a = []
conversion_list_group_a_2 = []
stats_for_group_a2 = []

while len(conversion_list_group_a)<3500: 
    user_id = random.randint(1, 53424)
    if user_id in already_checked: 
        continue
    else:
        already_checked.add(user_id)
        group_for_user = random.randint(0,1)
        if group_for_user == 0: 
            recommendatations = requests.post('http://127.0.0.1:5509/recommender', json={"model": "most_popular_model", "user_id":user_id})
            recommendatations = recommendatations.json()['result']
            true_books = list(test_df[(test_df.user_id==user_id)&(test_df.implicit_ratings==1)].book_id.values)
            conversion_for_user = len([book for book in recommendatations if book in true_books])/len(recommendatations)
            stats_for_user = [1 if book in true_books else 0 for book in recommendatations]
            stats_for_group_a.extend(stats_for_user)
            conversion_list_group_a.append(conversion_for_user)    
        else: 
            recommendatations = requests.post('http://127.0.0.1:5509/recommender', json={"model": "most_popular_model", "user_id":user_id})
            recommendatations = recommendatations.json()['result']
            true_books = list(test_df[(test_df.user_id==user_id)&(test_df.implicit_ratings==1)].book_id.values)
            conversion_for_user = len([book for book in recommendatations if book in true_books])/len(recommendatations)
            stats_for_user = [1 if book in true_books else 0 for book in recommendatations]
            stats_for_group_a2.extend(stats_for_user)
            conversion_list_group_a_2.append(conversion_for_user)    
            

print(np.mean(conversion_list_group_a), np.mean(conversion_list_group_a_2))

0.020830909393881734 0.02065405266513757


In [230]:
# конверсия будто бы не отличаетс, но применим стат критерий
from statsmodels.stats.proportion import test_proportions_2indep

result = test_proportions_2indep(count1=sum(stats_for_group_a), nobs1=len(stats_for_group_a), 
                                 count2=sum(stats_for_group_a2), nobs2=len(stats_for_group_a2))
result


<class 'statsmodels.stats.base.HolderTuple'>
statistic = 0.7670138533443746
pvalue = 0.4430732789440395
compare = 'diff'
method = 'agresti-caffo'
diff = 0.0004462309793531842
ratio = 1.023676104223762
odds_ratio = 1.0241418868117749
variance = 3.3872983429784293e-07
alternative = 'two-sided'
value = 0
tuple = (0.7670138533443746, 0.4430732789440395)

In [ ]:
# Видим что p-va.ue > 0.05 т.е не можем отклонить гипотезу о равенстве средних, в группах нет различий, теперь попробуем провести АВ тест 

In [231]:
# проведем AВ тест 

conversion_list_group_a = []
stats_for_group_a = []
conversion_list_group_b = []
stats_for_group_b = []

while len(conversion_list_group_a)<3500: 
    user_id = random.randint(1, 53424)
    if user_id in already_checked: 
        continue
    else:
        already_checked.add(user_id)
        group_for_user = random.randint(0,1)
        if group_for_user == 0: 
            recommendatations = requests.post('http://127.0.0.1:5509/recommender', json={"model": "most_popular_model", "user_id":user_id})
            recommendatations = recommendatations.json()['result']
            if len(recommendatations) == 0: 
                continue 
            else:
                true_books = list(test_df[(test_df.user_id==user_id)&(test_df.implicit_ratings==1)].book_id.values)
                conversion_for_user = len([book for book in recommendatations if book in true_books])/len(recommendatations)
                stats_for_user = [1 if book in true_books else 0 for book in recommendatations]
                stats_for_group_a.extend(stats_for_user)
                conversion_list_group_a.append(conversion_for_user)    
        else: 
            user_ind = int(np.where(user_index==user_id)[0][0])
            recommendatations = requests.post('http://127.0.0.1:5509/recommender', json={"model": "hybrid_model", 
                                                                                         "user_id":user_id, "usr_ind":user_ind})
            recommendatations = recommendatations.json()['result']
            if len(recommendatations) == 0: 
                continue 
            else: 
                true_books = list(test_df[(test_df.user_id==user_id)&(test_df.implicit_ratings==1)].book_id.values)
                conversion_for_user = len([book for book in recommendatations if book in true_books])/len(recommendatations)
                stats_for_user = [1 if book in true_books else 0 for book in recommendatations]
                stats_for_group_b.extend(stats_for_user)
                
                conversion_list_group_b.append(conversion_for_user)    
            

print(np.mean(conversion_list_group_a), np.mean(conversion_list_group_b))

0.020235321886830467 0.034489126064649774


In [232]:
result = test_proportions_2indep(count1=sum(stats_for_group_a), nobs1=len(stats_for_group_a), 
                                 count2=sum(stats_for_group_b), nobs2=len(stats_for_group_b))
result


<class 'statsmodels.stats.base.HolderTuple'>
statistic = -20.89526169339194
pvalue = 5.913170998006986e-97
compare = 'diff'
method = 'agresti-caffo'
diff = -0.015545653810050113
ratio = 0.5497838549988004
odds_ratio = 0.5410717172725447
variance = 5.53631641594952e-07
alternative = 'two-sided'
value = 0
tuple = (-20.89526169339194, 5.913170998006986e-97)

In [233]:
# Мы видим разницу и она стат значима, можно катить нашу версию рекомендаций в продакшн!